In [1]:
import ipywidgets
import threading
import time
from pydae.svg_tools import svg,grid2svg
import numpy as np
from IPython.display import SVG, display
import requests
import json

In [2]:
def draw_svg_circle(cx, cy, r, stroke_color='black', stroke_width=2, fill='none'):
    svg_code = f'<svg width="500" height="50">'
    svg_code += f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="{stroke_color}" stroke-width="{stroke_width}" fill="{fill}" />'
    svg_code += '</svg>'
    return svg_code

s = svg('pv_m_n_bess_db_reduced.svg')



# Example URL of the REST API endpoint that returns JSON data


def continuous_update():
    M = 2
    N = 3
    while True:
        # Make the request and get the JSON response
        api_url = "http://172.16.0.2:8000/measures"
        response = requests.get(api_url)
        data = response.json()

        

        s.set_text(f"P_LV0101",f"{data[f'p_s_LV0101']:5.2f} MW")
        s.set_text(f"P_LV0102",f"{data[f'p_s_LV0102']:5.2f} MW")
        NN = f'{N}'.zfill(2)
        s.set_text(f"P_LV01NN",f"{data[f'p_s_LV01{NN}']:5.2f} MW")        

        s.set_text(f"Q_LV0101",f"{data[f'q_s_LV0101']:5.2f} Mvar")
        s.set_text(f"Q_LV0102",f"{data[f'q_s_LV0102']:5.2f} Mvar")
        s.set_text(f"Q_LV01NN",f"{data[f'q_s_LV01{NN}']:5.2f} Mvar")   

        s.set_text(f"V_LV0101",f"{data[f'V_LV0101']:5.2f}  ")
        s.set_text(f"V_LV0102",f"{data[f'V_LV0102']:5.2f}  ")
        s.set_text(f"V_LV01NN",f"{data[f'V_LV01{NN}']:5.2f}  ")  
        
        MM = f'{M}'.zfill(2)
        s.set_text(f"P_LVMM01",f"{data[f'p_s_LV{MM}01']:5.2f} MW")
        s.set_text(f"P_LVMM02",f"{data[f'p_s_LV{MM}02']:5.2f} MW")
        s.set_text(f"P_LVMMNN",f"{data[f'p_s_LV{MM}{NN}']:5.2f} MW")        

        s.set_text(f"Q_LVMM01",f"{data[f'q_s_LV{MM}01']:10.2f} Mvar")
        s.set_text(f"Q_LVMM02",f"{data[f'q_s_LV{MM}02']:7.2f} Mvar")
        s.set_text(f"Q_LVMMNN",f"{data[f'q_s_LV{MM}{NN}']:7.2f} Mvar")  

        s.set_text(f"V_LVMM01",f"{data[f'V_LV{MM}01']:5.2f}")
        s.set_text(f"V_LVMM02",f"{data[f'V_LV{MM}02']:5.2f}")
        s.set_text(f"V_LVMMNN",f"{data[f'V_LV{MM}{NN}']:5.2f}")  

        P_POI_MW = data['p_line_POI_GRID']*100
        Q_POI_Mvar = data['q_line_POI_GRID']*100
        s.set_text(f"P_POI",f"{P_POI_MW:6.2f} MW")
        s.set_text(f"Q_POI",f"{Q_POI_Mvar:6.2f} Mvar")  

        json_data_POST_p_q_ref = {}
        json_data_POST_irrad = {}
        
        for i_m in range(1,M+1):
            
            name_j = "POI_MV"
            for i_n in range(1,N+1):
                name = f"{i_m}".zfill(2) + f"{i_n}".zfill(2)                
                # json_data_POST_p_q_ref.update({f"p_s_ppc_LV{name}":slider_p.value})
                # json_data_POST_p_q_ref.update({f"q_s_ppc_LV{name}":slider_q.value})
                json_data_POST_irrad.update({f"irrad_LV{name}":slider_I.value/1000})

        json_data_POST_p_q_ref.update({'P_POI_ref':slider_p.value*M*N/2})
        json_data_POST_p_q_ref.update({'Q_POI_ref':slider_q.value*M*N})

        s.set_text(f"V_POI",f"V = {data[f'V_POI']:3.2f}")
        html.value = s.tostring()

        time.sleep(0.05)  # Adjust the update interval as needed

        headers = {'Content-Type': 'application/json'}
        api_url = "http://172.16.0.4:8010/setpoints"
        response = requests.post(api_url, data=json.dumps(json_data_POST_p_q_ref), headers=headers)
        
        api_url = "http://172.16.0.2:8000/irrad"
        response = requests.post(api_url, data=json.dumps(json_data_POST_irrad), headers=headers)

        time.sleep(0.5)  # Adjust the update interval as needed

# Create an interactive slider widget
slider_p = ipywidgets.FloatSlider(min=0,  max=1.5, step=0.1, value=0, description='P:')
slider_q = ipywidgets.FloatSlider(min=-2, max=2, step=0.1, value=0, description='Q:')
slider_I = ipywidgets.FloatSlider(min=0, max=1100, step=10, value=1000, description='Irradiance:')
tab = ipywidgets.Tab()
tab.children = [slider_I, ipywidgets.VBox([slider_p,slider_q])]
tab.titles = ['Enviroment','References']

html = ipywidgets.HTML(value=s.tostring())

update_thread = threading.Thread(target=continuous_update)
update_thread.start()

ipywidgets.VBox([html,tab])

In [3]:
import requests
import json

def post_json(url, data):
    try:

        if response.status_code == 200:
            json_response = response.json()
            return json_response
        else:
            print(f"Error: {response.status_code} - {response.reason}")
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")

# Example URL of the REST API endpoint to post JSON data
api_url = "http://172.16.0.2:8001/setpoints"

# Example JSON data to post
json_data = {
"p_s_ppc_LV0101" : 0.6,
"p_s_ppc_LV0102" : 0.6,
"p_s_ppc_LV0103" : 0.6,
"p_s_ppc_LV0201" : 0.6,
"p_s_ppc_LV0202" : 0.6,
"p_s_ppc_LV0203" : 0.6,
"q_s_ppc_LV0101" : 0.5,
"q_s_ppc_LV0102" : 0.5,
"q_s_ppc_LV0103" : 0.5,
"q_s_ppc_LV0201" : 0.5,
"q_s_ppc_LV0202" : 0.5,
"q_s_ppc_LV0203" : 0.5
}

headers = {'Content-Type': 'application/json'}
response = requests.post(api_url, data=json.dumps(json_data), headers=headers)

# # Print the JSON response
# if json_response:
#     print(json_response)


In [4]:
model.report_z()

NameError: name 'model' is not defined